The following code is made for extracting data from mbox file (obtained from gmail) and filtering it by applied patterns. Then only cleaned data will be saved into csv format.
1. First you need to extract on your PC mbox files from zip folder obtained from google takeout
2. You need to upload mbox.file into this environement(one by one, each mbox.file has to be processed separately).To upload it click on  Files icon
in the left pane of this file (the last vertical icon).
3. When your file will be uploaded, hover over it and you will see 3 dots, click on them and choose "copy path"
4. Insert copied data in the code field "insert here copied info"(you can find it using CTRL+F)
5. By default filtering patterns are set to None, it means that the code will only extract info about Date/Subject/From/To/Body of message and convert it to csv format in its raw form with all sensitive data.
6. Before indicating cleaning patterns you might want to see how it looks in original. To do it, after doing 4 step, run the following code, by clicking on the play icon, at the begging of the code. When the process will be finished, you will see the green mark near to the play icon. It means that all data is saved in file under the name cleanedmails.csv . You will find this file in the same menu where your first uploaded mbox.file is stored.
7. To open cleanedmails.csv you can double click on it or dowlnload it on your PC. If the file is too large, it can not be opened directly by double click, so you will need to download it. To download the file, hover over it, click on 3 dots menu and select option "download"
8. If you want to start applying your cleaning patterns, go to the end of the code.
9. You will find code fields with names filter_include/filter_exlude, here you can write patterns for such columns of data as Date/Subject/From/To but not yet avalaible for column Body of message.
10. You can indicate as many patters as you need. If you don't want to set one of filters (to include or to exclude), please set it like: filter_include = []/filter_exclude = [] The needed format of patters is {'Date': 'Enter your text'}, {'Subject': 'Enter your text'}, {'From': 'Enter your text'}, {'To': 'Enter your text'}.
11. If you want to set a number of patterns: filter_exclude = [ {'Subject': 'Enter your text'}, {'From': 'Enter your text'}]
12. After listing all required patters, run the code by clicking on the "play icon"
12. Every time you will apply your changes and run the code cleanedmails.csv file will be overwritten.
13. To check the result of your changes, open the file cleanedmails.csv by double click or download it on your PC.

**NOTE: during the process of extracting data from mbox files utf-8 decoding error may happen. It means that in stead of normal text you will see a number of characters.**



In [ ]:
import mailbox
import csv

def getcharsets(msg):
    charsets = set({})
    for c in msg.get_charsets():
        if c is not None:
            charsets.update([c])
    return charsets

def handleerror(errmsg, emailmsg,cs):
    print()
    print(errmsg)
    print("This error occurred while decoding with ",cs," charset.")
    print("These charsets were found in the one email.",getcharsets(emailmsg))
    print("This is the subject:",emailmsg['subject'])
    print("This is the sender:",emailmsg['From'])


def getbodyfromemail(msg):
    body = None
    #Walk through the parts of the email to find the text body.
    if msg.is_multipart():
        for part in msg.walk():

            # If part is multipart, walk through the subparts.
            if part.is_multipart():

                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        # Get the subpart payload (i.e the message body)
                        body = subpart.get_payload(decode=True)
                        #charset = subpart.get_charset()

            # Part isn't multipart so get the email body
            elif part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True)
                #charset = part.get_charset()

    # If this isn't a multi-part message then get the payload (i.e the message body)
    elif msg.get_content_type() == 'text/plain':
        body = msg.get_payload(decode=True)

   # No checking done to match the charset with the correct part.
    for charset in getcharsets(msg):
        try:
            body = body.decode(charset)
        except UnicodeDecodeError:
            return ''
        except AttributeError:
            return ''
    return body


def filter_data_include(message, filters):
    for filter in filters:
        for (k, f) in filter.items():
            if k not in message or f not in message[k]:
                return False
    return True


def filter_data_exclude(message, filters):
    for filter in filters:
        for (k, f) in filter.items():
            if k in message and f in message[k]:
                return False
    return True



def write_csv(filters_include=None, filters_exclude=None):
    writer = csv.writer(open("wp5.csv", 'w', newline='', encoding='utf-8'))
    for message in mailbox.mbox("put_mbox_file_here.mbox"):
        if filters_include and not filter_data_include(message, filters_include):
            continue
        if filters_exclude and not filter_data_exclude(message, filters_exclude):
            continue

        writer.writerow([
                message['Date'],
                message['Subject'],
                message['From'],
                message['To'],
                getbodyfromemail(message)] )

###Fields for patterns are below###
#{"From": "Jessica"}
filter_include = []
filter_exclude = []

write_csv(filter_include, filter_exclude)